# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,-3.77,76,21,4.43,US,1738719305
1,1,kenora,49.8167,-94.4337,-18.46,65,75,3.09,CA,1738719414
2,2,fort bragg,35.1390,-79.0060,14.91,72,0,0.00,US,1738719416
3,3,longyearbyen,78.2186,15.6401,-3.09,68,40,13.38,SJ,1738719417
4,4,aykhal,66.0000,111.5000,-28.77,80,100,2.76,RU,1738719418


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 25) & (city_data_df["Max Temp"] > 19)]

city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,nova vicosa,-17.8919,-39.3719,24.55,88,0,3.62,BR,1738719501
74,74,kapa'a,22.0752,-159.3190,24.18,57,0,8.23,US,1738719545
81,81,guanica,17.9716,-66.9080,24.17,50,0,2.46,PR,1738719553
105,105,lihue,21.9789,-159.3672,23.77,53,0,8.23,US,1738719581
120,120,wailua homesteads,22.0669,-159.3780,23.44,57,0,8.23,US,1738719598
143,143,badarganj,25.6740,89.0549,19.20,49,0,0.47,BD,1738719625
191,191,cabo san lucas,22.8909,-109.9124,22.17,49,0,4.12,MX,1738719644
197,197,coahuayana de hidalgo,18.7000,-103.6583,22.65,91,0,0.45,MX,1738719686
216,216,saucillo,28.0167,-105.2833,21.13,12,0,2.56,MX,1738719707
248,248,salalah,17.0151,54.0924,20.05,21,0,4.12,OM,1738719744


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,sikasso,ML,10.9167,-7.0000,22.0,
11,kidal,ML,18.4411,1.4078,29.0,
12,blackmans bay,AU,-43.0167,147.3167,45.0,
16,margaret river,AU,-33.9500,115.0667,44.0,
36,queenstown,ZA,-31.8976,26.8753,45.0,
...,...,...,...,...,...,...
535,centenario,AR,-38.8043,-68.1400,41.0,
536,prince rupert,CA,54.3161,-130.3201,21.0,
562,villamontes,BO,-21.2500,-63.5000,41.0,
572,sucre,BO,-19.0333,-65.2627,48.0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    # print(name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sikasso - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
queenstown - nearest hotel: No hotel found
kataba - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
guanica - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
badarganj - nearest hotel: No hotel found
tumayr - nearest hotel: No hotel found
taybad - nearest hotel: No hotel found
colesberg - nearest hotel: No hotel found
flagstaff - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
rikaze - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points('Lng', 'Lat', 
    geo=True, tiles = True, alpha=0.70, marker='o', 
    frame_width = 700, frame_height = 500,
    s='Humidity', c='City', 
    hover_alpha=1, 
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)